In [11]:
import os
import math
import pickle
import numpy as np
import matplotlib.pyplot as plt

# ───────── CONFIG ─────────
DATA_DIR       = 'data_for_training'
OUTPUT_IMG_DIR = 'image_examples/highres'
FS             = 2048
window_idx     = 0        # which 200 ms slice to show
cols           = 2        # two gestures per row
dpi            = 300      # high resolution

# ensure output directory exists
os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)

# load segmented windows
with open(os.path.join(DATA_DIR, 'segmented_200ms.pkl'), 'rb') as f:
    seg200 = pickle.load(f)

# determine reference vmin/vmax from first subject
all_subjects = sorted(seg200.keys())
ref_subj     = all_subjects[0]
gestures_ref = sorted(seg200[ref_subj].keys())

# gather reference values
all_vals_ref = np.hstack([
    seg200[ref_subj][gest][window_idx].ravel()
    for gest in gestures_ref
])
vmin = np.percentile(all_vals_ref, 1)
vmax = np.percentile(all_vals_ref, 99)

print(f"Reference scale from subject {ref_subj}: vmin={vmin:.3f}, vmax={vmax:.3f}")

# ───────── LOOP OVER ALL SUBJECTS ─────────
for subj in all_subjects:
    gestures = sorted(seg200[subj].keys())
    rows = math.ceil(len(gestures) / cols)

    fig, axes = plt.subplots(
        rows, cols,
        figsize=(cols*6, rows*5),
        constrained_layout=True
    )
    axes = axes.flatten()

    for idx, gest in enumerate(gestures):
        ax = axes[idx]
        data = seg200[subj][gest][window_idx]
        im = ax.imshow(
            data,
            aspect='auto',
            origin='upper',
            vmin=vmin,
            vmax=vmax
        )
        ax.set_title(f"{subj}/{gest}", fontsize=10)
        ax.set_xlabel("Sample", fontsize=8)
        ax.set_ylabel("Channel", fontsize=8)

        # draw separators every 8 channels, offset +0.5
        for y in range(8, data.shape[0], 8):
            ax.axhline(y + 0.5, color='white', linewidth=1)

    # remove any unused subplots
    for j in range(len(gestures), rows * cols):
        fig.delaxes(axes[j])

    # single shared colorbar
    cbar = fig.colorbar(
        im,
        ax=axes[:len(gestures)].tolist(),
        orientation='vertical',
        fraction=0.04, pad=0.02
    )
    cbar.ax.tick_params(labelsize=6)

    plt.suptitle(f"{subj}: 200 ms Window #{window_idx+1} Across Gestures\n"
                 f"(uniform scale from {ref_subj})", fontsize=12)

    # save high‑res PNG
    out_path = os.path.join(OUTPUT_IMG_DIR, f"{subj}_200ms_windows.png")
    fig.savefig(out_path, dpi=dpi, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved: {out_path}")

Reference scale from subject HS1: vmin=-22.728, vmax=22.166
Saved: image_examples/highres\HS1_200ms_windows.png
Saved: image_examples/highres\HS2_200ms_windows.png
Saved: image_examples/highres\HS3_200ms_windows.png
Saved: image_examples/highres\HS4_200ms_windows.png
Saved: image_examples/highres\HS5_200ms_windows.png
Saved: image_examples/highres\HS6_200ms_windows.png
Saved: image_examples/highres\HS7_200ms_windows.png
Saved: image_examples/highres\HS8_200ms_windows.png
